In [1]:
import requests
import pandas as pd
import io

# APIs

An Application Programming Interface, or API, is a structured way to retrieve data from a website. Using an API is safer and easier than something like webscraping, since what you get back is already in a usable format. Many organizations use APIs like:
- Government organizations ([US Government](https://www.data.gov/developers/apis))
- Large companies ([Twitter API](https://developer.twitter.com/en/docs))
- News organizations ([NYT API](https://developer.nytimes.com/))
- And [many more](https://github.com/public-apis/public-apis)

If you type `how to use an api in python` in google, you get back many articles walking through how to use an API. It is a well documented and useful tool to be familiar with.

The CDC has a large number of datasets avaiable at https://data.cdc.gov/, which can be accessed using their API. 

Let's look at the Vaccine Hesitancy for COVID-19: County and local estimates dataset, located at https://data.cdc.gov/Vaccinations/Vaccine-Hesitancy-for-COVID-19-County-and-local-es/q9mh-h2tw.

If you browse to this page, you should see a button in the upper right labeled "API". Click this button and then change the value in the bottom right from JSON to CSV. Copy the API Endpoint.

In [3]:
URL = 'https://data.cdc.gov/resource/q9mh-h2tw.csv'

To interact with this we'll use the `requests` library.

In [4]:
response = requests.get(URL)

Once we have this response, we can check the status code. A 200 indicates success.

In [10]:
response.status_code

200

To take the result, we need to use the _io_ library to get it into a format that can be read into a _pandas_ DataFrame.

In [11]:
vaccine_hesitancy = pd.read_csv(io.StringIO(response.text))

In [12]:
vaccine_hesitancy

,fips_code,county_name,state,estimated_hesitant,estimated_hesitant_or_unsure,estimated_strongly_hesitant,social_vulnerability_index,svi_category,ability_to_handle_a_covid,cvac_category,...,percent_hispanic,percent_non_hispanic_american,percent_non_hispanic_asian,percent_non_hispanic_black,percent_non_hispanic_native,percent_non_hispanic_white,geographical_point,state_code,county_boundary,state_boundary
0,1123,"Tallapoosa County, Alabama",ALABAMA,0.1806,0.2400,0.1383,0.89,Very High Vulnerability,0.64,High Concern,...,0.0242,0.0022,0.0036,0.2697,0.0000,0.6887,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-85.841259 33.104456, -85.8409...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
1,1121,"Talladega County, Alabama",ALABAMA,0.1783,0.2350,0.1368,0.87,Very High Vulnerability,0.84,Very High Concern,...,0.0229,0.0043,0.0061,0.3237,0.0003,0.6263,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-86.303069 33.46316, -86.30306...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
2,1131,"Wilcox County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.93,Very High Vulnerability,0.94,Very High Concern,...,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.52534299999999 32.132773, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
3,1129,"Washington County, Alabama",ALABAMA,0.1735,0.2357,0.1337,0.73,High Vulnerability,0.82,Very High Concern,...,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-88.45317899999999 31.505388, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
4,1133,"Winston County, Alabama",ALABAMA,0.1805,0.2313,0.1379,0.70,High Vulnerability,0.80,High Concern,...,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370,POINT (-86.844516 32.756889),AL,"MULTIPOLYGON (((-87.63656399999999 34.120908, ...","MULTIPOLYGON (((-88.139988 34.581703, -88.1352..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,20003,"Anderson County, Kansas",KANSAS,0.1560,0.2068,0.0963,0.23,Low Vulnerability,0.63,High Concern,...,0.0189,0.0209,0.0001,0.0050,0.0000,0.9484,POINT (-98.38018 38.484729),KS,MULTIPOLYGON (((-95.36024599999999 38.39013699...,"MULTIPOLYGON (((-102.045212 38.697567, -102.04..."
996,19029,"Cass County, Iowa",IOWA,0.1323,0.1905,0.0795,0.20,Very Low Vulnerability,0.09,Very Low Concern,...,0.0270,0.0024,0.0028,0.0020,0.0014,0.9529,POINT (-93.500061 42.074659),IA,"MULTIPOLYGON (((-95.155284 41.331877999999996,...","MULTIPOLYGON (((-96.49877599999999 42.561043, ..."
997,19171,"Tama County, Iowa",IOWA,0.1293,0.1877,0.0781,0.33,Low Vulnerability,0.10,Very Low Concern,...,0.0978,0.0723,0.0046,0.0092,0.0000,0.8075,POINT (-93.500061 42.074659),IA,MULTIPOLYGON (((-92.76746299999999 42.21013999...,"MULTIPOLYGON (((-96.49877599999999 42.561043, ..."
998,19085,"Harrison County, Iowa",IOWA,0.1323,0.1905,0.0795,0.06,Very Low Vulnerability,0.02,Very Low Concern,...,0.0174,0.0004,0.0049,0.0019,0.0004,0.9620,POINT (-93.500061 42.074659),IA,"MULTIPOLYGON (((-96.07862899999999 41.761963, ...","MULTIPOLYGON (((-96.49877599999999 42.561043, ..."


**Question:** What do you notice about the results?

This returned only 1000 results. It is common for APIs to limit the number of results returned.

Click on the API Docs button and let's see if we can figure out how to get more results.

If you click on the "Paging through Data" link, you'll find how to control this. It seems that by default, this API will only return 1000 results, but we can change this by using the `$limit` query parameter.

One way to include a query parameter is to pass it as a dictionary to the `params` argument of `.get()`.

In [13]:
params = {
    '$limit': 5000
}

In [14]:
response = requests.get(URL, params = params)

vaccine_hesitancy = pd.read_csv(io.StringIO(response.text))
vaccine_hesitancy

What if we don't care about the whole country and only want to get the results for Tennessee?

To accomplish this, we can filter by specifying the state parameter.

In [16]:
params = {
    'state': 'TENNESSEE'
}

In [17]:
response = requests.get(URL, params = params)

vaccine_hesitancy = pd.read_csv(io.StringIO(response.text))
vaccine_hesitancy

,fips_code,county_name,state,estimated_hesitant,estimated_hesitant_or_unsure,estimated_strongly_hesitant,social_vulnerability_index,svi_category,ability_to_handle_a_covid,cvac_category,...,percent_hispanic,percent_non_hispanic_american,percent_non_hispanic_asian,percent_non_hispanic_black,percent_non_hispanic_native,percent_non_hispanic_white,geographical_point,state_code,county_boundary,state_boundary
0,47155,"Sevier County, Tennessee",TENNESSEE,0.1411,0.2073,0.0884,0.61,High Vulnerability,0.53,Moderate Concern,...,0.0599,0.0039,0.0078,0.0090,0.0002,0.8999,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-83.784379 35.876163, -83.7841...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."
1,47165,"Sumner County, Tennessee",TENNESSEE,0.1221,0.1799,0.0754,0.21,Low Vulnerability,0.63,High Concern,...,0.0488,0.0025,0.0140,0.0727,0.0004,0.8412,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-86.690759 36.409742, -86.6909...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."
2,47175,"Van Buren County, Tennessee",TENNESSEE,0.1382,0.2064,0.0879,0.60,High Vulnerability,0.97,Very High Concern,...,0.0071,0.0000,0.0000,0.0028,0.0000,0.9571,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-85.597422 35.726036, -85.5982...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."
3,47167,"Tipton County, Tennessee",TENNESSEE,0.1337,0.2022,0.0837,0.45,Moderate Vulnerability,0.66,High Concern,...,0.0278,0.0029,0.0067,0.1836,0.0000,0.7569,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-89.86997199999999 35.582524, ...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."
4,47189,"Wilson County, Tennessee",TENNESSEE,0.1169,0.1696,0.0722,0.15,Very Low Vulnerability,0.56,Moderate Concern,...,0.0423,0.0029,0.0164,0.0712,0.0002,0.8497,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-86.5809 36.208819, -86.580764...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,47105,"Loudon County, Tennessee",TENNESSEE,0.1260,0.1874,0.0786,0.39,Low Vulnerability,0.46,Moderate Concern,...,0.0878,0.0013,0.0072,0.0092,0.0000,0.8769,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-84.388333 35.768682999999996,...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."
91,47147,"Robertson County, Tennessee",TENNESSEE,0.1408,0.2067,0.0884,0.43,Moderate Vulnerability,0.62,High Concern,...,0.0693,0.0023,0.0056,0.0719,0.0006,0.8326,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-87.073559 36.427653, -87.0811...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."
92,47169,"Trousdale County, Tennessee",TENNESSEE,0.1519,0.2256,0.0968,0.52,Moderate Vulnerability,0.73,High Concern,...,0.0108,0.0053,0.0024,0.1018,0.0000,0.8416,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-86.268898 36.450441999999995,...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."
93,47091,"Johnson County, Tennessee",TENNESSEE,0.1400,0.2081,0.0885,0.57,Moderate Vulnerability,0.85,Very High Concern,...,0.0203,0.0051,0.0048,0.0345,0.0000,0.9229,POINT (-86.343226 35.842998),TN,"MULTIPOLYGON (((-81.951482 36.473863, -81.9539...","MULTIPOLYGON (((-82.222064 36.15696, -82.22427..."


**Your Turn:** Find another dataset available from the CDC that looks interesting to you and utilize the API to retrieve it.